## Data Collection

In [1]:
# including the project directory to the notebook level
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import modules
from alumni_scripts import data_process as dp
import json

In [ ]:
# run a single update of the data pull
with open('../auths.json', 'r') as fp:
    api_args = json.load(fp)

time_args = {
    'start_year': 2018,'start_month': 8,'start_day': 1,'start_hour': 0,'start_minute': 0,'start_second': 0,
    'end_year': 2018,'end_month': 8,'end_day': 1,'end_hour': 0,'end_minute': 30,'end_second': 0,
    'trend_id': '2681',
    'save_path':'../data/raw_data/alumni_data.csv'
}
api_args.update(time_args)

dp.pull_offline_data(**api_args)

In [2]:
# download data in a loop
time_args = [
    {
    'start_year': 2018,'start_month': 7,'start_day': 1,'start_hour': 0,'start_minute': 0,'start_second': 0,
    'end_year': 2018,'end_month': 12,'end_day': 31,'end_hour': 23,'end_minute': 59,'end_second': 59,
    'trend_id': '2681',
    'save_path':'../data/raw_data/alumni_data_jul2dec2018.csv'
    },
    {
    'start_year': 2019,'start_month': 1,'start_day': 1,'start_hour': 0,'start_minute': 0,'start_second': 0,
    'end_year': 2019,'end_month': 6,'end_day': 30,'end_hour': 23,'end_minute': 59,'end_second': 59,
    'trend_id': '2681',
    'save_path':'../data/raw_data/alumni_data_jan2jun2019.csv'  
    },
    {
    'start_year': 2019,'start_month': 7,'start_day': 1,'start_hour': 0,'start_minute': 0,'start_second': 0,
    'end_year': 2019,'end_month': 12,'end_day': 31,'end_hour': 23,'end_minute': 59,'end_second': 59,
    'trend_id': '2681',
    'save_path':'../data/raw_data/alumni_data_jul2dec2019.csv'  
    },
    {
    'start_year': 2020,'start_month': 1,'start_day': 1,'start_hour': 0,'start_minute': 0,'start_second': 0,
    'end_year': 2020,'end_month': 6,'end_day': 15,'end_hour': 23,'end_minute': 59,'end_second': 59,
    'trend_id': '2681',
    'save_path':'../data/raw_data/alumni_data_jan2jun2020.csv'  
    }
]
for i in time_args:
    with open('../auths.json', 'r') as fp:
        api_args = json.load(fp)
    api_args.update(i)
    dp.pull_offline_data(**api_args)
    print("Success!")

Success!
Success!
Success!
Success!


## Create Offline Batch Time Series Data Base for Alumni Hall data

In [12]:
# close client
client.close()

In [1]:
# including the project directory to the notebook level
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import modules
from alumni_scripts import data_process as dp
from alumni_scripts import alumni_data_utils as utils
import json
import pandas as pd
from influxdb import DataFrameClient

### Collate the data

In [2]:
# collate batch of data
file_names = ['jul2dec2018', 'jan2jun2019', 'jul2dec2019', 'jan2jun2020']
dflist = []
for fname in file_names:
    df_ = pd.read_csv('../data/raw_data/alumni_data_{}.csv'.format(fname))
    df_['time'] = pd.to_datetime(df_['time'])
    df_.set_index(keys='time',inplace=True, drop = True)
    dflist.append(df_)
df = utils.mergerows(dflist)

### Create meta data

In [3]:
# Create column_aliases
d1 = {'column_names': list(df.columns)}
column_aliases = [
    'pchwst', 'vrf50', 'oat', 'sat', 'sat_stpt', 'oah', 'vrf67', 'pchw_flow',
    'hwe', 'vrf1', 'vrf30', 'vrf34', 'vrf74', 'cwe', 'hws_st_stpt', 'vrf60',
    'vrf63', 'hws_st', 'hws_vlv1', 'vrf77', 'vrf64', 'vrf10', 'ee', 'hws_rt',
    'vrf100', 'vrf40', 'hws_flow', 'vrf108', 'vrf20'
]

# Create column stats
d2 = {}
for i, j in zip(df.columns, column_aliases):
    d2.update({j:i})
d1['column_aliases'] = d2
stats = {}
d3 = dict(df.describe())
for key in d3.keys():
    stats[key] = dict(d3[key])
d1['column_stats'] = stats

# Create meta data json file
with open('../alumni_scripts/meta_data.json', 'w') as fp:
    json.dump(d1, fp, indent=4)

### Remove outliers

In [4]:
df_cleaned = dp.offline_batch_data_clean(meta_data_path='../alumni_scripts/meta_data.json', df = df)

In [5]:
df_cleaned.columns = column_aliases

### Push data to a database

In [6]:
"""
before the next steps launch influxd client at a cli
sudo influxd
"""
# launch python client for influxdb
client = DataFrameClient(host='localhost', port=8086)
# create a database inc case it's not there
client.create_database('bdx_batch_db')
# get list of database
client.get_list_database()
# switch to the databaase you want
client.switch_database('bdx_batch_db')
# write "dataframe" as "measurements"
client.write_points(dataframe=df_cleaned,
                    measurement='alumni_data_v1',
                    tags={
                        'data_cleaned': 'True',
                        'aggregated': False,
                        'time-interval': '5 minutes'
                    },
                    protocol='line',
                    batch_size=5000)
# see measurement added to curent db
client.get_list_measurements()
client.close()

[{'name': '_internal'}, {'name': 'bdx_batch_db'}]

### Read data from the database

In [19]:
"""
before the next steps launch influxd client at a cli
sudo influxd
"""
# launch python client for influxdb
client = DataFrameClient(host='localhost', port=8086)
# switch to the databaase you want
client.switch_database('bdx_batch_db')
results_obj = client.query(
    "select ee,oat,sat,sat_stpt from alumni_data_v1 \
    where time >= '2018-11-15 12:00:00' \
    and time < '2018-11-15 12:25:00'"
)
df2 = results_obj['alumni_data_v1']
df2

,ee,oat,sat,sat_stpt
2018-11-15 12:00:00+00:00,0.000000,33.107929,70.612915,73.0
2018-11-15 12:05:00+00:00,0.000000,33.107929,70.612915,73.0
2018-11-15 12:10:00+00:00,46.622402,33.107929,70.612915,73.0
2018-11-15 12:15:00+00:00,0.000000,33.107929,70.612915,73.0
2018-11-15 12:20:00+00:00,0.000000,33.107929,70.612915,73.0


In [1]:
# including the project directory to the notebook level
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import modules
from alumni_scripts import data_process as dp
from alumni_scripts import alumni_data_utils as utils
import json
import pandas as pd
from influxdb import DataFrameClient

In [2]:
df = pd.read_csv('../data/raw_data/alumni_data_jul2dec2018.csv',)
df['time'] = pd.to_datetime(df['time'])
df.set_index(keys='time',inplace=True, drop = True)
df_cleaned = dp.offline_batch_data_clean(meta_data_path='../alumni_scripts/meta_data.json', df = df)

In [3]:
"""
before the next steps launch influxd client at a cli
sudo influxd
"""
# launch python client for influxdb
client = DataFrameClient(host='localhost', port=8086)
# create a database inc case it's not there
client.create_database('demo_alumni')
# get list of database
client.get_list_database()
# switch to the databaase you want
client.switch_database('demo_alumni')
# write "dataframe" as "measurements"
client.write_points(dataframe=df_cleaned, measurement='alumni_jul2dec2018', protocol='line', batch_size=5000)
# see measurement added to curent db
client.get_list_measurements()

In [8]:
results_obj = client.query(
    "select * from alumni_jul2dec2018 where time >= '2018-11-15 12:00:00' and time < '2018-11-15 12:05:00'"
)
df2 = results_obj['alumni_jul2dec2018']
df2

,AHU_1 outdoorAirTemp,AHU_1 supplyAirTemp,AHU_1 supplyAirTempSetpoint,Alumni_Hall_PCHWS_T value,Alumni_Hall_PCHW_Flow value,CAFE_103_Indoor_Unit_50 localSetpoint,CHW_BTU_METER currentKbtuDeltaReading,CLASS_ROOM_201_Indoor_Unit_100 localSetpoint,COORIDOR_006A_Indoor_Unit_34 localSetpoint,CORRIDOR_108_Indoor_Unit_60 localSetpoint,...,Hot_Water_Flow_AI_2 value,LOUNGE_100_Indoor_Unit_40 localSetpoint,MEETING_117A_Indoor_Unit_63 localSetpoint,MEMORIAL_HALL_202_Indoor_Unit_67 localSetpoint,OFFICE_115_Indoor_Unit_77 localSetpoint,OFFICE_305_Indoor_Unit_74 localSetpoint,READING_ROOM_206_Indoor_Unit_108 localSetpoint,TUTORING_010_Indoor_Unit_10 localSetpoint,WRITING_STUDIO_113_Indoor_Unit_20 localSetpoint,WeatherDataProfile humidity
2018-11-15 12:00:00+00:00,33.107929,70.612915,73.0,42.969219,-0.047234,72.0,0.0,72.0,72.0,72.0,...,28.971073,72.0,72.0,72.0,75.0,72.0,72.0,72.0,72.0,85.0


In [10]:
results_obj2 = client.query(
    "select * from alumni_jul2dec2018 where time = '2018-11-15 12:00:00'"
)
df3 = results_obj2['alumni_jul2dec2018']
df3

,AHU_1 outdoorAirTemp,AHU_1 supplyAirTemp,AHU_1 supplyAirTempSetpoint,Alumni_Hall_PCHWS_T value,Alumni_Hall_PCHW_Flow value,CAFE_103_Indoor_Unit_50 localSetpoint,CHW_BTU_METER currentKbtuDeltaReading,CLASS_ROOM_201_Indoor_Unit_100 localSetpoint,COORIDOR_006A_Indoor_Unit_34 localSetpoint,CORRIDOR_108_Indoor_Unit_60 localSetpoint,...,Hot_Water_Flow_AI_2 value,LOUNGE_100_Indoor_Unit_40 localSetpoint,MEETING_117A_Indoor_Unit_63 localSetpoint,MEMORIAL_HALL_202_Indoor_Unit_67 localSetpoint,OFFICE_115_Indoor_Unit_77 localSetpoint,OFFICE_305_Indoor_Unit_74 localSetpoint,READING_ROOM_206_Indoor_Unit_108 localSetpoint,TUTORING_010_Indoor_Unit_10 localSetpoint,WRITING_STUDIO_113_Indoor_Unit_20 localSetpoint,WeatherDataProfile humidity
2018-11-15 12:00:00+00:00,33.107929,70.612915,73.0,42.969219,-0.047234,72.0,0.0,72.0,72.0,72.0,...,28.971073,72.0,72.0,72.0,75.0,72.0,72.0,72.0,72.0,85.0


In [11]:
# drop the database after the demo
client.drop_database('demo_alumni')
client.get_list_database()

[{'name': '_internal'}]

## Create meta_data : Demo

In [1]:
# including the project directory to the notebook level
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import modules
import pandas as pd
import json
import numpy as np

### Create column_aliases

In [2]:
df = pd.read_csv('../data/raw_data/alumni_data_jul2dec2018.csv', index_col='time')

d1 = {'column_names': list(df.columns)}
column_aliases = [
    'pchwst', 'vrf50', 'oat', 'sat', 'sat_stpt', 'oah', 'vrf67', 'pchw_flow',
    'hwe', 'vrf1', 'vrf30', 'vrf34', 'vrf74', 'cwe', 'hws_st_stpt', 'vrf60',
    'vrf63', 'hws_st', 'hws_vlv1', 'vrf77', 'vrf64', 'vrf10', 'ee', 'hws_rt',
    'vrf100', 'vrf40', 'hws_flow', 'vrf108', 'vrf20'
]

d2 = {}
for i, j in zip(df.columns, column_aliases):
    d2.update({j:i})
d1['column_aliases'] = d2

### Create column stats

In [3]:
stats = {}
d3 = dict(df.describe())
for key in d3.keys():
    stats[key] = dict(d3[key])
d1['column_stats'] = stats

### Dump meta data

In [4]:
with open('../alumni_scripts/meta_data.json', 'w') as fp:
    json.dump(d1, fp, indent=4)

### Read meta data

In [5]:
with open('../alumni_scripts/meta_data.json', 'r') as fp:
        meta_data_ = json.load(fp)
meta_data = meta_data_.copy()
for key, value in meta_data_['column_stats'].items():
    if value['std'] == 0:
        meta_data['column_stats'][key]['std'] = 0.0001  # add small std for constant values
stats = pd.DataFrame(meta_data['column_stats'])

## Plot data before and after cleaning: Demo

In [1]:
# including the project directory to the notebook level
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import modules
from alumni_scripts import data_process as dp
from alumni_scripts import alumni_data_utils as utils
import json
import pandas as pd

In [2]:
df = pd.read_csv('../data/raw_data/alumni_data_jul2dec2018.csv',)
df['time'] = pd.to_datetime(df['time'])
df.set_index(keys='time',inplace=True, drop = True)

In [3]:
df_cleaned = dp.offline_batch_data_clean(meta_data_path='../alumni_scripts/meta_data.json', df = df)

In [ ]:
for col_name in df.columns:
    utils.dataframeplot(df[[col_name]],lazy=False,legend=True)
    utils.dataframeplot(df_cleaned[[col_name]],lazy=False,legend=True)

## Create Time Series Data Base: Demo only